# Домашнее задание 3. 

## Предсказание пользовательской оценки отеля по тексту отзыва.

Мы собрали для вас отзывы по 1500 отелям из совершенно разных уголков мира. Что это за отели - секрет. Вам дан текст отзыва и пользовательская оценка отеля. Ваша задача - научиться предсказывать оценку отеля по отзыву. Данные можно скачать [тут](https://www.kaggle.com/c/hseds-texts-2020/data?select=train.csv).

Главная метрика - Mean Absolute Error (MAE). Во всех частях домашней работы вам нужно получить значение MAE не превышающее 1. В противном случае мы будем вынуждены не засчитать задание :( 

Для измерения качества вашей модели используйте разбиение данных на train и test и замеряйте качество на тестовой части.

#### Про данные:
Каждое ревью состоит из двух текстов: positive и negative - плюсы и минусы отеля. В столбце score находится оценка пользователя - вещественное число 0 до 10. Вам нужно извлечь признаки из этих текстов и предсказать по ним оценку.

Удачи! 💪

Использовать внешние данные для обучения строго запрещено. Можно использовать предобученные модели из torchvision.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np

PATH_TO_TRAIN_DATA = '/content/drive/MyDrive/train.csv'

In [ ]:
import string

import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')

from nltk.tokenize import word_tokenize

ps = PorterStemmer()

def process_text(text):
    if len(text.split(' ')) == 0:
        return 'empty empty empty'
    words = []
    for word in word_tokenize(text.lower()):
        if word not in string.punctuation:
            #word = ps.stem(word)
            words.append(word)
    return words

In [ ]:
import pandas as pd

df = pd.read_csv(PATH_TO_TRAIN_DATA)
df['opinion'] = df['positive'].str.cat(df['negative'], sep =" ")
df = df.drop('review_id', axis=1)
print(df.shape)
df.head()

(100000, 4)


,negative,positive,score,opinion
0,There were issues with the wifi connection,No Positive,7.1,No Positive There were issues with the wifi c...
1,TV not working,No Positive,7.5,No Positive TV not working
2,More pillows,Beautiful room Great location Lovely staff,10.0,Beautiful room Great location Lovely staff ...
3,Very business,Location,5.4,Location Very business
4,Rooms could do with a bit of a refurbishment ...,Nice breakfast handy for Victoria train stati...,6.7,Nice breakfast handy for Victoria train stati...


Предобработка текста может сказываться на качестве вашей модели.
Сделаем небольшой препроцессинг текстов: удалим знаки препинания, приведем все слова к нижнему регистру. 
Однако можно не ограничиваться этим набором преобразований. Подумайте, что еще можно сделать с текстами, чтобы помочь будущим моделям? Добавьте преобразования, которые могли бы помочь по вашему мнению.

Также мы добавили разбиение текстов на токены. Теперь каждая строка-ревью стала массивом токенов.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, random_state=42)

### Часть 1. 1 балл

Обучите логистическую регрессию на TF-IDF векторах текстов.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import mean_absolute_error
from scipy import sparse
import numpy as np

In [ ]:
tfidf = TfidfVectorizer(max_features=2000, tokenizer=process_text)

positive_train = tfidf.fit_transform(df_train['positive'])
positive_test = tfidf.transform(df_test['positive'])
negative_train = tfidf.fit_transform(df_train['negative'])
negative_test = tfidf.transform(df_test['negative'])

x_train = sparse.hstack((positive_train, negative_train))
y_train = df_train['score']
x_test = sparse.hstack((positive_test, negative_test))
y_test = df_test['score']

In [ ]:
df_train.head()

,negative,positive,score,opinion
98980,Cups washed up each day they stayed dirty all...,No Positive,4.2,"[no, positive, cups, washed, up, each, day, th..."
69824,Tiny but tidy cosy pool,Good location friendly staff,8.8,"[good, location, friendly, staff, tiny, but, t..."
9928,No Negative,everything good location really nice room,8.3,"[everything, good, location, really, nice, roo..."
75599,The walls between the rooms are a bit thin Yo...,The linen was always fresh Never felt like an...,10.0,"[the, linen, was, always, fresh, never, felt, ..."
95621,Nothing,Excellent location beside the underground,9.6,"[excellent, location, beside, the, underground..."


In [ ]:
model = Ridge(alpha=5, max_iter=50)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
error = mean_absolute_error(y_test, y_pred) 
print('MAE Ridge:', round(error, 6))

MAE Ridge: 0.821561


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train.mul(10).astype(int))
y_pred = model.predict_proba(x_test)
y_pred = (y_pred * model.classes_).sum(axis=1) / 10
error = mean_absolute_error(y_test, y_pred) 
print('MAE LogReg:', round(error, 6))

INFO - 08:02:34: NumExpr defaulting to 2 threads.


MAE LogReg: 0.80854


### Часть 2. 3 балла

Обучите логистическую регрессию на усредненных Word2Vec векторах. 

In [ ]:
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
import logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
EMB_SIZE = 500

w2v_model = Word2Vec(min_count=1,
                     window=2,
                     size=EMB_SIZE,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=4)

In [ ]:
w2v_model.build_vocab(df['opinion'], progress_per=50000)
w2v_model.train(df['opinion'], total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.wv.most_similar(positive=["good"])

[('great', 0.7073086500167847),
 ('excellent', 0.6422793865203857),
 ('nice', 0.5858272910118103),
 ('decent', 0.5412341952323914),
 ('poor', 0.508098840713501),
 ('very', 0.4963108003139496),
 ('comfortable', 0.4706449508666992),
 ('superb', 0.46917957067489624),
 ('plentiful', 0.4450289309024811),
 ('tasty', 0.44377583265304565)]

In [ ]:
def embed_mean(model, data, emb):
    df_vectors = np.zeros((1, emb))
    sentences_vectors = np.zeros((1, emb))
    for sentence in data:
        sent_vectors = np.zeros_like((2, emb))
        w = 0
        for word in sentence:
            
            try:
                if w == 0:
                    word_vector = np.array(model[word])
                    sent_vectors = word_vector
                else:
                    word_vector = np.array(model[word])
                    sent_vectors = np.vstack((sent_vectors, word_vector))
                w += 1
            except:
                w += 1
                continue
        try:
            if sentences_vectors.shape[0] == 1:
                sentences_vectors = np.mean(sent_vectors, axis=0)
            else:
                sentences_vectors = np.vstack((sentences_vectors, np.mean(sent_vectors, axis=0)))
        except:
            sentences_vectors = np.vstack((sentences_vectors, np.zeros((1, emb))))
        
        if sentences_vectors.shape[0] > 2500:
            if df_vectors.shape[0] == 1:
                df_vectors = sentences_vectors
            else:
                df_vectors = np.vstack((df_vectors, sentences_vectors))
            sentences_vectors = np.zeros((1, emb))

    if df_vectors.shape[0] < data.shape[0]:
        df_vectors = np.vstack((df_vectors, sentences_vectors))

    return df_vectors

In [ ]:
mean_df = embed_mean(w2v_model, df['opinion'], EMB_SIZE)
mean_df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(100000, 500)

In [ ]:
y = df['score']
x_train, x_test, y_train, y_test = train_test_split(mean_df, y, random_state=42)

model = LogisticRegression(penalty='l2', max_iter=2000, C=1)
model.fit(x_train, y_train.mul(10).astype(int))
y_pred = model.predict_proba(x_test)
y_pred = (y_pred * model.classes_).sum(axis=1) / 10
error = mean_absolute_error(y_test, y_pred) 
print('MAE LogReg on embeddings:', round(error, 6))

MAE LogReg on embeddings: 0.962973


Усредняя w2v вектора, мы предполагаем, что каждое слово имеет равноценный вклад в смысл предложения, однако это может быть не совсем так. Теперь попробуйте воспользоваться другой концепцией и перевзвесить слова при получении итогового эмбеддинга текста. В качестве весов используйте IDF (Inverse document frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv(PATH_TO_TRAIN_DATA)
df['opinion'] = df['positive'].str.cat(df['negative'], sep =" ")

tfidf = TfidfVectorizer(tokenizer=process_text)

_ = tfidf.fit_transform(df['opinion'])

df['opinion'] = df['opinion'].apply(process_text)

In [ ]:
def embed_idf(model, data, tfidf_w, tfidf_i, emb):
    df_vectors = np.zeros((1, emb))
    sentences_vectors = np.zeros((1, emb))
    for sentence in data:
        sent_vectors = np.zeros_like((2, emb))

        idf_arr = np.array([])
        w = 0
        for word in sentence:
            
            try:
                index = tfidf_w.index(word)
                idf_arr = np.append(idf_arr, tfidf_i[index])
                if w == 0:
                    word_vector = np.array(model[word])
                    sent_vectors = word_vector
                else:
                    word_vector = np.array(model[word])
                    sent_vectors = np.vstack((sent_vectors, word_vector))
                w += 1
            except:
                w += 1
                continue
        try:
            idf_arr = (idf_arr / np.sum(idf_arr)).T
            idf_arr = np.reshape(idf_arr, (idf_arr.shape[0], 1))
            if sentences_vectors.shape[0] == 1:
                sentences_vectors = np.sum(sent_vectors * idf_arr, axis=0)
            else:
                sentences_vectors = np.vstack((sentences_vectors, np.sum(sent_vectors * idf_arr, axis=0)))
        except:
            sentences_vectors = np.vstack((sentences_vectors, np.zeros((1, emb))))
        
        if sentences_vectors.shape[0] > 2500:
            if df_vectors.shape[0] == 1:
                df_vectors = sentences_vectors
            else:
                df_vectors = np.vstack((df_vectors, sentences_vectors))
            sentences_vectors = np.zeros((1, emb))

    if df_vectors.shape[0] < 100000:
        df_vectors = np.vstack((df_vectors, sentences_vectors))

    return df_vectors

In [ ]:
tfidf_i = tfidf.idf_
tfidf_w = list(tfidf.vocabulary_.keys())

idf_df = embed_idf(w2v_model, df['opinion'], tfidf_w, tfidf_i, EMB_SIZE)
idf_df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(100000, 500)

In [ ]:
y = df['score']
x_train, x_test, y_train, y_test = train_test_split(idf_df, y, random_state=42)

model = LogisticRegression(penalty='l2', max_iter=2000, C=1)
model.fit(x_train, y_train.mul(10).astype(int))
y_pred = model.predict_proba(x_test)
y_pred = (y_pred * model.classes_).sum(axis=1) / 10
error = mean_absolute_error(y_test, y_pred) 
print('MAE LogReg on weighted embeddings:', round(error, 6))

MAE LogReg on weighted embeddings: 0.962061


Проведите эксперименты с размерностью эмбеддинга. Для каждого из двух методов постройте график зависимости качества модели от размерности эмбеддинга. 
#### Сделайте выводы:
**Чем больше размерность эмбединга, тем качество модели лучше. При этом усредненные и взвешенные вектора показывают примерно одинаковые результаты**

Теперь попробуйте обучить логистическую регрессию на любых других эмбеддингах размерности 300 и сравните качество с Word2Vec.
#### Выводы:

### Часть 3. 6 баллов

Теперь давайте воспользуемся более продвинутыми методами обработки текстовых данных, которые мы проходили в нашем курсе. Обучите RNN/Transformer для предсказания пользовательской оценки. Получите ошибку меньше, чем во всех вышеперечисленных методах.

Если будете обучать RNN, попробуйте ограничить максимальную длину предложения. Некоторые отзывы могут быть слишком длинные относительно остальных.

Чтобы пользоваться DataLoader, все его элементы должны быть одинаковой размерности. Для этого вы можете добавить нулевой паддинг ко всем предложениям (см пример pad_sequence)

In [5]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from torch import autograd
import spacy
import torch.optim as optim
import spacy
from torchtext.data import Field, TabularDataset, BucketIterator
import pandas as pd

In [6]:
df = pd.read_csv(PATH_TO_TRAIN_DATA)
df['opinion'] = df['positive'].str.cat(df['negative'], sep =" ")
df = df.drop(['review_id', 'positive', 'negative'], axis=1)

df_train, df_test = train_test_split(df, random_state=42)
df_train.to_csv('/content/drive/MyDrive/TrainSet.csv', index=False)
df_test.to_csv('/content/drive/MyDrive/TestSet.csv', index=False)

In [7]:
spacy_en = spacy.load("en")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def tokenize(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


opinion = Field(sequential=True, use_vocab=True, tokenize=tokenize, lower=True)
score = Field(sequential=False, use_vocab=False, is_target=True, dtype=torch.float)

fields = {"opinion": ("text", opinion), "score": ("score", score)}

train_data, test_data = TabularDataset.splits(path='/content/drive/MyDrive/', train='TrainSet.csv', test='TestSet.csv', format='csv', fields=fields)

opinion.build_vocab(train_data, max_size=20000, min_freq=5)

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), batch_size=256, device=device,
    sort_within_batch = True, sort_key = lambda x: len(x.text))

In [8]:
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers)
        self.fc_out = nn.Linear(hidden_size, 1)
        

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device)

        embedded = self.embedding(x)
        outputs, _ = self.rnn(embedded, (h0, c0))
        prediction = self.fc_out(outputs[-1, :, :])

        return prediction

In [9]:
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
import sys

def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model = model.to(device).train()
    model.train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    all_mae = np.array([])
    for batch_idx, batch in enumerate(train_dataloader):
        data = batch.text.to(device=device)
        targets = batch.score.to(device=device)

        scores = model(data)
        loss = criterion(scores.squeeze(1), targets.type_as(scores))

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        error = mean_absolute_error(targets.cpu().detach().numpy(), scores.cpu().detach().numpy()).mean()

        total_loss += loss.item()
        all_mae = np.append(all_mae, error)
        num_batches += 1
        all_losses.append(loss.detach().item())

    loss = total_loss / num_batches
    return loss, np.mean(all_mae)


def predict(model, test_dataloader, criterion, device="cuda:0"):
    model = model.to(device).eval()
    total_loss = 0
    num_batches = 0
    all_mae = np.array([])
    for batch_idx, batch in enumerate(test_dataloader):
        data = batch.text.to(device=device)
        targets = batch.score.to(device=device)

        scores = model(data)
        loss = criterion(scores.squeeze(1), targets.type_as(scores))
        error = mean_absolute_error(targets.cpu().detach().numpy(), scores.cpu().detach().numpy()).mean()

        total_loss += loss.item()
        all_mae = np.append(all_mae, error)
        num_batches += 1
    losses =  total_loss / num_batches
    return losses, np.mean(all_mae)


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model = model.to(device)
    loss_on_train = np.array([])
    loss_on_val = np.array([])
    mae_on_train = np.array([])
    mae_on_val = np.array([])
    for epoch in range(n_epochs):
        print('\n#############################EPOCH ', epoch + 1, '#############################', sep='')
        loss1, mae1 = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
        mae_on_train = np.append(mae_on_train, mae1)
        loss_on_train = np.append(loss_on_train, loss1)

        with torch.no_grad():
            loss2, mae2 = predict(model, val_dataloader, criterion, device)
            mae_on_val = np.append(mae_on_val, mae2)
            loss_on_val = np.append(loss_on_val, loss2)
            
        print('Train loss: ', round(loss1, 6), 'Val loss: ', round(loss2, 6), 'Train MAE: ', round(mae1, 4), 'Val MAE: ', round(mae2, 4))

        if scheduler is not None:
            scheduler.step()

In [11]:
input_size = len(opinion.vocab)
hidden_size = 512
num_layers = 2
embedding_size = 300
num_epochs = 10

model = RNN_LSTM(input_size, embedding_size, hidden_size, num_layers).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.05, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.7)
criterion = nn.MSELoss()

In [12]:
train(model, train_iterator, test_iterator, criterion, optimizer, scheduler=scheduler)


#############################EPOCH 1#############################
Train loss:  3.744079 Val loss:  3.235187 Train MAE:  1.4486 Val MAE:  1.5578

#############################EPOCH 2#############################
Train loss:  2.781396 Val loss:  2.857127 Train MAE:  1.3187 Val MAE:  1.2287

#############################EPOCH 3#############################
Train loss:  2.639892 Val loss:  2.331853 Train MAE:  1.2806 Val MAE:  1.165

#############################EPOCH 4#############################
Train loss:  2.40022 Val loss:  2.370409 Train MAE:  1.2125 Val MAE:  1.2379

#############################EPOCH 5#############################
Train loss:  2.21228 Val loss:  2.134879 Train MAE:  1.1495 Val MAE:  1.0977

#############################EPOCH 6#############################
Train loss:  1.752652 Val loss:  1.539142 Train MAE:  1.0053 Val MAE:  0.934

#############################EPOCH 7#############################
Train loss:  1.435641 Val loss:  1.762264 Train MAE:  0.9015 Val MA

### Бонус. 10 баллов

Побейте качество 0.75 в [соревновании](https://www.kaggle.com/c/hseds-texts-2020/leaderboard). Можете воспользоваться вышеперечисленными методами или попробовать что-нибудь еще.

### Полезные материалы
1) https://github.com/aladdinpersson/Machine-Learning-Collection

2) https://fasttext.cc/docs/en/supervised-tutorial.html

3) https://stackoverflow.com/questions/61213493/pytorch-lstm-for-multiclass-classification-typeerror-not-supported-between

4) https://www.analyticsvidhya.com/blog/2020/01/first-text-classification-in-pytorch/